In [24]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import requests
from random import *
from tqdm import tqdm #progress bar
from datetime import datetime
import os

In [12]:
make_model_data = pd.read_csv('data/make_and_model_links.csv')
make_model_data

,car_make,id1,car_model,id2,link
0,Mercedes-Benz,17200,190,126,https://suchen.mobile.de/fahrzeuge/search.html...
1,Mercedes-Benz,17200,200,127,https://suchen.mobile.de/fahrzeuge/search.html...
2,Mercedes-Benz,17200,220,128,https://suchen.mobile.de/fahrzeuge/search.html...
3,Mercedes-Benz,17200,230,129,https://suchen.mobile.de/fahrzeuge/search.html...
4,Mercedes-Benz,17200,240,130,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...
2230,Wiesmann,25650,MF 30,4,https://suchen.mobile.de/fahrzeuge/search.html...
2231,Wiesmann,25650,MF 35,6,https://suchen.mobile.de/fahrzeuge/search.html...
2232,Wiesmann,25650,MF 4,7,https://suchen.mobile.de/fahrzeuge/search.html...
2233,Wiesmann,25650,MF 5,8,https://suchen.mobile.de/fahrzeuge/search.html...


In [13]:
random_number = randrange(len(make_model_data))
print("make: ", make_model_data['car_make'][random_number], " model: ", make_model_data['car_model'][random_number])
print(make_model_data['link'][random_number])

make:  Audi  model:  R8
https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=1900;29&ref=quickSearch&sfmr=false&vc=Car


Function to scrape one make and model combination:

In [32]:
def scrape_links_for_one_make_model(make_model_input_link = make_model_link, sleep = 1, make_model_input_data = make_model_data, save_to_csv = True):

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2} # this is to not load images
    chrome_options.add_experimental_option("prefs", prefs)

    #start a driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

    #get the number of pages
    driver.get(make_model_input_link)
    make_model_link_lastpage_source = driver.page_source
    make_model_link_soup = BeautifulSoup(make_model_link_lastpage_source, 'html.parser')

    last_button = make_model_link_soup.findAll('span', {'class': 'btn btn--muted btn--s'})
    try:
        print("This many pages found: ", last_button[len(last_button)-1].text)
        last_button_number = last_button[len(last_button)-1].text
        last_button_number = int(last_button_number)
    except:
        last_button_number = int(1)
    
    driver.close()

    #start scraping the ads
    
    links_on_multiple_pages = []

    for i in tqdm(range(1, last_button_number + 1)):

        # start a new driver every time
        # we need this to avoid getting blocked by the website if we don't do this, we will get captcha

        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.images": 2} # this is to not load images
        chrome_options.add_experimental_option("prefs", prefs)

        #start a driver
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

        #we need to navigate to the page
        one_page_link = make_model_input_link + "&pageNumber=" + str(i)

        driver.get(one_page_link)
        time.sleep(sleep)
        base_source = driver.page_source
        base_soup = BeautifulSoup(base_source, 'html.parser')

        #get all the links
        cars_add_list_all = base_soup.findAll('a', {'class': re.compile('^link--muted no--text--decoration')})

        links_on_one_page = []

        for i in range(len(cars_add_list_all)):

            link = cars_add_list_all[i]['href']
            
            if not link.endswith('SellerAd'):
                # filter out links that end with 'SellerAd' (these are links to ads and we do not need them)
                links_on_one_page.append(link)

        for elements in links_on_one_page:
            links_on_multiple_pages.append(elements)

        driver.close() #close the driver

    links_on_one_page_df = pd.DataFrame({'ad_link' : links_on_multiple_pages})
    links_on_one_page_df['make_model_link'] = make_model_input_link #via this we can see which make and model the links belong to
    
    #datetime string
    now = datetime.now() 
    datetime_string = str(now.strftime("%Y%m%d_%H%M%S"))

    links_on_one_page_df['download_date_time'] = datetime_string

    #check is the make and model is in the dataframe
    if isinstance(make_model_input_data, pd.DataFrame):
        #join the dataframes to get make and model information
        links_on_one_page_df = pd.merge(links_on_one_page_df, make_model_input_data, how = 'left', left_on= ['make_model_link'], right_on = ['link'])

    #save the dataframe if save_to_csv is True
    if save_to_csv:
        #check if folder exists and if not create it
        if not os.path.exists('data/make_model_ads_links'):
            os.makedirs('data/make_model_ads_links')

        links_on_one_page_df.to_csv(str('data/make_model_ads_links/links_on_one_page_df' + datetime_string + '.csv'), index = False)

    return(links_on_one_page_df)

In [17]:
link_on_multiple_pages_data = scrape_links_for_one_make_model(
    make_model_input_link = 'https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=22900;13&ref=quickSearch&sfmr=false&vc=Car',
    make_model_input_data = make_model_data,
    save_to_csv=True)

#This will output a lots of strings. I do not know how to do this silently. If you know how to do this, please let me know.
#But otherwise, you can use the following code to get the links



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-16-0e24163b87b2>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  0%|          | 0/30 [00:00<?, ?it/s]

This many pages found:  30




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-16-0e24163b87b2>:35: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  3%|▎         | 1/30 [00:07<03:46,  7.82s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  7%|▋         | 2/30 [00:27<06:48, 14.60s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 10%|█         | 3/30 [00:37<05:43, 12.72s/it]

====== WebDriver man

In [20]:
link_on_multiple_pages_data

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
614,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
615,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
616,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
617,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...


# From here we just need to create a loop that goes throug all the make and model combinations

In [22]:
def multiple_link_on_multiple_pages_data(make_model_input_links = [], sleep = 1, make_model_input_data = make_model_data, save_to_csv = True):

    make_model_data = pd.DataFrame()

    for one_make_model_link in make_model_input_links:
        
        one_page_adds = scrape_links_for_one_make_model(make_model_input_link = one_make_model_link, sleep = sleep, make_model_input_data = make_model_input_data, save_to_csv = save_to_csv)
        make_model_data = pd.concat([make_model_data, one_page_adds], ignore_index=True)


In [30]:
#generate 5 random numbers between 1 1000
random_numbers = np.random.randint(0, len(make_model_data), 5)
random_numbers
print(make_model_data['link'][random_numbers])

403     https://suchen.mobile.de/fahrzeuge/search.html...
121     https://suchen.mobile.de/fahrzeuge/search.html...
765     https://suchen.mobile.de/fahrzeuge/search.html...
523     https://suchen.mobile.de/fahrzeuge/search.html...
1084    https://suchen.mobile.de/fahrzeuge/search.html...
Name: link, dtype: object


In [33]:
multiple_link_on_multiple_pages_data(make_model_input_links = make_model_data['link'][random_numbers])



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-32-7cfac9390674>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  0%|          | 0/1 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-32-7cfac9390674>:39: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
100%|██████████| 1/1 [00:09<00:00,  9.90s/it]


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.

This many pages found:  15




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  7%|▋         | 1/15 [00:20<04:44, 20.34s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 13%|█▎        | 2/15 [00:30<03:06, 14.31s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 20%|██        | 3/15 [00:40<02:30, 12.55s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found

This many pages found:  45


  0%|          | 0/45 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  2%|▏         | 1/45 [00:09<07:13,  9.85s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  4%|▍         | 2/45 [00:19<06:55,  9.66s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
  7%|▋         | 3/45 [00:30<07:11, 10.27s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win3

In [ ]:
# concatenate the dataframes in one folder to get one file (with different columns)
